In [1]:
import pandas as pd
from sklearn import datasets
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.metrics import mean_squared_error, r2_score
import pickle

In [2]:
df = pd.read_csv(r'dataset.csv', on_bad_lines='skip', delimiter=';', low_memory = False, decimal = ",")

# print(df.head(5))
print(df.shape) # dimensiones de la tabla de datos

# tipología variables

df["churn"] = df["churn"].astype("bool") # object, int64
#df[df.columns[52:54]] = df[df.columns[52:54]].astype("string")
#df[df.columns[54]] = df[df.columns[54]].astype("factor")
df = df.drop(["Customer_ID"], axis=1)


(100000, 100)


In [3]:
df = df.drop(columns=['area','prizm_social_one','dualband','refurb_new','hnd_webcap','infobase','HHstatin','dwllsize','ethnic',
                        'kid0_2','kid3_5','kid6_10','kid11_15','kid16_17','creditcd'])

print(df.shape)

(100000, 84)


In [4]:
# eliminamos nans
df = df.dropna()
print(df.shape)

# identificar outliers

# def find_outliers_IQR(df):
#   q1 = df.quantile(0.25)
#   q3 = df.quantile(0.75)
#     IQR = q3-q1
#  outliers = df[((df<(q1-1.5*IQR)) | (df>(q3+1.5*IQR)))]
#   return outliers

# df.loc[find_outliers_IQR(df).index,:]


(41830, 84)


In [5]:
# dummy
var_x_dummy = ['new_cell','crclscod','asl_flag','ownrent','dwlltype','marital']
df = pd.get_dummies(df, columns = var_x_dummy)

In [6]:
# balanceo
print(df.churn.value_counts())

False    21378
True     20452
Name: churn, dtype: int64


In [7]:
df.info()
print(df.shape)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41830 entries, 0 to 99993
Columns: 141 entries, rev_Mean to marital_U
dtypes: bool(1), float64(69), int64(8), uint8(63)
memory usage: 27.4 MB
(41830, 141)


In [11]:
X_entrena, X_valida, Y_entrena, Y_valida = train_test_split(df.drop(columns = "churn"), df['churn'],train_size=.70)
print("Estructura de datos de entrenamiento... ", X_entrena.shape)


Estructura de datos de entrenamiento...  (29280, 140)
       rev_Mean  mou_Mean  totmrc_Mean  da_Mean  ovrmou_Mean  ovrrev_Mean  \
43705   68.4400    924.75      59.9900   1.7325        53.25      15.9750   
80928   41.7225    123.75      49.9900   1.7325         0.00       0.0000   
79536   31.7225    181.00      39.9900   1.7325         0.00       0.0000   
63404   33.1400     15.75      39.9900   0.0000         0.00       0.0000   
50251   33.3650    720.25      10.0000   0.9900        85.25      21.3125   
...         ...       ...          ...      ...          ...          ...   
25455   22.5300     71.50      22.4925   0.0000         0.00       0.0000   
22168   17.6075     23.00      16.9900   0.0000         1.25       0.4625   
19088   38.7425    523.00      44.9900   0.0000         0.00       0.0000   
7846    12.8125     37.75      10.0000   0.0000         0.00       0.0000   
50190   31.8850    162.00      29.9900   0.4950         4.00       1.4000   

       vceovr_Mean  d

In [9]:
model_rf = RandomForestClassifier(n_estimators = 150)
model_rf.fit(X_entrena, Y_entrena)

RandomForestClassifier(n_estimators=150)

In [10]:
# Almacenar modelo en pickle

pickle.dump(model_rf, open('trading_model.p', 'wb'))

In [45]:
loaded_model = pickle.load(open('trading_model.p', 'rb'))
predicciones = loaded_model.predict(X_valida)

In [46]:
comparaciones = pd.DataFrame(Y_valida)
# comparaciones = comparaciones.assign(Sales_Real = Y_valida)
comparaciones = comparaciones.assign(Predicho = predicciones.flatten().tolist())
print(comparaciones)

       churn  Predicho
39470  False      True
41149   True     False
46304   True     False
54263   True      True
42050  False     False
...      ...       ...
7094   False     False
27287   True      True
70649  False     False
80745  False     False
69502  False     False

[12550 rows x 2 columns]


In [49]:
mat_confusion = confusion_matrix(
                    y_true    = Y_valida,
                    y_pred    = predicciones
                )

accuracy = accuracy_score(
            y_true    = Y_valida,
            y_pred    = predicciones,
            normalize = True
           )

print("Matriz de confusión")
print("-------------------")
print(mat_confusion)
print("")
print(f"El accuracy de test es: {100 * accuracy} %")

Matriz de confusión
-------------------
[[4090 2357]
 [2479 3624]]

El accuracy de test es: 61.46613545816733 %


In [ ]:
# balanceo = True
# tipo_balanceo = 1

# if balanceo:
#     X_dum = X.copy()
#     idx = X.index
#     X_dum['idx'] = idx
#     X_dum.head()
# 
#    Y_dum = y.copy()

    # Balanceos
#     from imblearn.under_sampling import RandomUnderSampler
#     from imblearn.over_sampling import RandomOverSampler
#     from imblearn.over_sampling import SMOTE
#     from imblearn.combine import SMOTEENN
#     from imblearn.combine import SMOTETomek